# Code for running inference

10-18-2024

To convert this to a script, should have a name for the whole training task which goes in dataset.json and/or datalist.json because this will help for the task.json needed for inference.

Look into the minimal amount of metaata monai requires in the form of a task.json or other struct to initialize the builder and then run the ensemble


In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform

import mri_data
import monai_training

from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

from reload_recursive import reload_recursive

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, filter_first_ses  # noqa: E402, F401
from monai_training import preprocess  # noqa: E402


Setup logging


In [3]:
log_dir = ".logs"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logger.add(
    os.path.join(log_dir, "file_{time:%Y_%m_%d}.log"), rotation="6h", level="DEBUG"
)

1

**Edit These**


In [4]:
#! Set these variables
work_dir_name = "choroid_pineal_pituitary2"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal_pituitary2_pred"
task_name = "infer_choroid_pineal_pituitary"
modalities = ["flair", "t1"]
save_dir = Path("/mnt/h/3Tpioneer_bids_predictions")

Set paths


In [7]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair.t1_choroid_pineal_pituitary2_pred.nii.gz


In [ ]:
if not train_dataset_file.is_file():
    dataset = preprocess.parse_datalist(work_dir / "training-datalist.json", dataroot)
    preprocess.save_dataset(dataset, train_dataset_file)

2024-11-10 17:25:20.473 | INFO     | monai_training.preprocess:parse_datalist:282 - Loading /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/training-datalist.json
2024-11-10 17:25:20.476 | INFO     | monai_training.preprocess:parse_datalist:283 - /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary2/training-datalist.json exists: True


In [9]:
def inference_exists(dataset: DataSet) -> DataSet:
    count = 0
    dataset_new = DataSet(dataset.dataroot)
    for scan in dataset:
        if not (save_dir / scan.relative_path / prediction_filename).is_file():
            dataset_new.append(scan)
        else:
            count += 1
    logger.info(f"{count} scans already have inference")
    return dataset_new

### Do Preparation

Get all the scans that were not in the training and testing set to create the inference dataset


In [10]:
# the scans that were used in the training
dataset_train, _ = preprocess.load_dataset(train_dataset_file)

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(
    dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses, inference_exists]
)
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ["subid", "sesid"])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

2024-11-10 17:25:49.474 | INFO     | __main__:inference_exists:9 - 0 scans already have inference


Prepare the images in the inference dataset (i.e combine flair and t1 images)


In [11]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(["flair", "t1"])
dataset_proc.dataset.sort(key=lambda s: s.subid)

2024-11-10 17:25:59.217 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
  0%|          | 0/547 [00:00<?, ?it/s]2024-11-10 17:25:59.234 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1229, sesid=20170715)
2024-11-10 17:25:59.307 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2085, sesid=20170130)
  9%|▉         | 49/547 [00:00<00:01, 480.91it/s]2024-11-10 17:25:59.376 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz


Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated


 22%|██▏       | 122/547 [00:25<01:27,  4.86it/s]

/usr/local/fsl/share/fsl/bin/fslmerge: line 2: 228476 Aborted                 (core dumped) /usr/local/fsl/bin/fslmerge "$@"


 67%|██████▋   | 366/547 [00:26<00:04, 44.06it/s]2024-11-10 17:26:25.790 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz


Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated


 83%|████████▎ | 453/547 [00:52<00:12,  7.27it/s]2024-11-10 17:26:52.261 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=2001, sesid=20161222)


/usr/local/fsl/share/fsl/bin/fslmerge: line 2: 228609 Aborted                 (core dumped) /usr/local/fsl/bin/fslmerge "$@"


100%|██████████| 547/547 [00:53<00:00, 10.28it/s]


Save the datalist and task files


In [13]:
images = []
for scan in dataset_proc.dataset[:20]:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

2024-11-10 17:27:48.637 | INFO     | __main__:<module>:6 - Will run inference on 20 scans


In [14]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [15]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/20 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

2024-11-10 17:31:09,632 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:   5%|▌         | 1/20 [02:59<56:52, 179.60s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is depre

2024-11-10 17:34:09,391 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...:  10%|█         | 2/20 [05:59<53:54, 179.70s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.a

...

2024-11-10 17:53:49,006 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1015/ses-20190326/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...:  45%|████▌     | 9/20 [25:38<29:23, 160.31s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.a

2024-11-10 17:56:46,058 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1016/ses-20200421/flair.t1_choroid_pineal_pituitary2_pred.nii.gz


Ensembling (rank 0)...:  50%|█████     | 10/20 [28:36<27:35, 165.57s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.

KeyboardInterrupt: 